# Haystack Pinecone simple test
WARNING: version issues...  
pinecone-haystack needs pinecone-client<3
pinecone-client<3 not compatible with pinecone-datasets (requires v3)

In [1]:
import os
from pathlib import Path
from getpass import getpass

In [2]:
from haystack.document_stores.types import DuplicatePolicy
from haystack import Document
from haystack import Pipeline
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder
from haystack_integrations.components.retrievers.pinecone import PineconeEmbeddingRetriever
from haystack_integrations.document_stores.pinecone import PineconeDocumentStore

### PineCone setup:

*Double check*: if you have a free-tier pinecone account, you are limited to one index, and a cap on docs (5k??)

In [3]:
import torch
torch.cuda.is_available()

True

In [5]:
#os.environ["PINECONE_API_KEY"] = "YOUR_PINECONE_API_KEY"
pak = "PINECONE_API_KEY"
if pak in os.environ:
    print("Found PINECONE_API_KEY in os.environ, no need to prompt")
else: 
    os.environ["PINECONE_API_KEY"]= getpass("Enter PINECONE API key:")

idx_name = "my-index"


Found PINECONE_API_KEY in os.environ, no need to prompt


_todo_: check for existing index, decide to reuse or replace

In [8]:
# pinecone-client v3 to 4??
# from pinecone import Pinecone, ServerlessSpec

# pc = Pinecone(
#     api_key=os.environ.get("PINECONE_API_KEY")
# )

# # Now do stuff
# if idx_name in pc.list_indexes().names():
#     print(f"Found index '{idx_name}' already defined, leaving it as is...")
# else:
#     print(f"Did not find index '{idx_name}' already defined, CREATE it now...")
#     pc.create_index(
#         name=idx_name, 
#         dimension=1536, 
#         metric='euclidean',
#         spec=ServerlessSpec(
#             cloud='aws',
#             region='us-west-2'
#         )
#     )

In [7]:
# newer api... use new/fresh env...
# from pinecone import Pinecone, ServerlessSpec
# pc = Pinecone(api_key="675fd35e-96c1-4c62-9dbc-320be0193133")

In [6]:
from haystack_integrations.document_stores.pinecone import PineconeDocumentStore

# Make sure you have the PINECONE_API_KEY environment variable set
document_store = PineconeDocumentStore(metric="cosine", dimension=768, index=idx_name, environment = "YOUR_ENVIRONMENT")

MaxRetryError: HTTPSConnectionPool(host='controller.your_environment.pinecone.io', port=443): Max retries exceeded with url: /databases (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f8cbd912440>: Failed to resolve 'controller.your_environment.pinecone.io' ([Errno -2] Name or service not known)"))

In [ ]:
# document_store = PineconeDocumentStore(index=idx_name, namespace="barnsp", dimension=768)
# document_store

In [ ]:
documents = [Document(id='1', content="There are over 7,000 languages spoken around the world today."),
             Document(id='2', content="The lips of the glasses were coated in salt."),
             Document(id='3', content="Mustangs from the '60s are some of my favorite models."),
             
             Document(id='4', content="Large language model (LLM) work can be very confusing."),
             Document(id='5', content="French is one of the tongues I have studied."),
             Document(id='6', content="Groovy is more comfortable for me than Java is (or was)."),
             Document(id='7', content="The cool kids love Python; I am not cool."),

             Document(id='8', content="Elephants have been observed to communicate in a an intelligent manner."),
             Document(id='9', content="Cats have been observed to NGAF."),

             Document(id='10', content="I need to study Spanish before I return to Spain."),

             Document(id='11', content="In certain places, you can witness the phenomenon of bioluminescent waves.")
            ]

In [ ]:
document_embedder = SentenceTransformersDocumentEmbedder()
document_embedder.warm_up()
documents_with_embeddings = document_embedder.run(documents)

In [ ]:
document_store.write_documents(documents_with_embeddings.get("documents"), policy=DuplicatePolicy.OVERWRITE)

In [ ]:
query_pipeline = Pipeline()

query_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder())
query_pipeline.add_component("retriever", PineconeEmbeddingRetriever(document_store=document_store, 
                                                                     top_k=5))

query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")

### Search for languages

In [ ]:
query = "How many languages are there?"
res = query_pipeline.run({"text_embedder": {"text": query}})
res

In [ ]:
assert res['retriever']['documents'][0].content == "There are over 7,000 languages spoken around the world today."

### Search For animals

In [ ]:
query = "What animals are there?"

res = query_pipeline.run({"text_embedder": {"text": query}})
res